# МГТУ им. Н.Э.Баумана | ИУ5 | 6 семестр | ТМО | ЛР№5

https://github.com/ugapanyuk/courses_current/wiki/LAB_TMO_ENSEMBLES_1

**Задание:**

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.

4. Обучите следующие ансамблевые модели:
    - две модели группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
    - AdaBoost;
    - градиентный бустинг.
5. Оцените качество моделей с помощью одной из подходящих для задачи метрик Сравните качество полученных моделей.

## Теория

**Регрессия** - задача прогнозирования количественных переменных на основе данных. В основе регрессионного анализа лежит построение функциональной зависимости между одной или несколькими независимыми переменными и одной зависимой переменной.

**Классификация** — задача прогнозирования категориальных переменных на основе данных. Задача классификации заключается в назначении объекта одному из заранее заданных классов на основе его характеристик.

Бэггинг и случайный лес - https://nbviewer.org/github/ugapanyuk/courses_current/blob/main/notebooks/ensembles/bagging_forest.ipynb

Бустинг - https://nbviewer.jupyter.org/github/ugapanyuk/courses_current/blob/main/notebooks/ensembles/boosting.ipynb


Идея использования ансамблевых моделей состоит в том, что оценивается результат совместного голосования различных моделей. Варианты объединения моделей в ансамбль могут быть различными и могут использоваться для решения различных задач.

### Бэггинг

Бэггинг позволяет снизить дисперсию (variance) обучаемого классификатора, уменьшая величину, на которую ошибка будет отличаться, если обучать модель на разных наборах данных, или другими словами, предотвращает переобучение.


Пусть имеется обучающая выборка X
 размера N
. С помощью бутстрэпа сгенерируем из этой выборки K
 подвыборок X1
 ... XK
. Теперь на каждой выборке обучим свою модель Mi(x)
. Итоговый классификатор будет усреднять ответы всех моделей (выбирать средний результат голосования):


Важно то что модель бэггинга строится на ансамбле разных обучающих выборок, но одинаковых моделей.

### Случайный лес

Случайный лес можно рассматривать как алгоритмом бэггинга над решающими деревьями.

Но при этом каждое решающее дерево строится на случайно выбранном подмножестве признаков. Эта особенность называется "feature bagging" и основана на методе случайных подпространств.

### Сверхслучайные деревья

В сверхслучайных деревьях (Extremely Randomized Trees) больше случайности в том, как вычисляются разделения в узлах.





## Практика

Датасет: https://github.com/ongaunjie1/credit-score-prediction


### Загрузка и первичный анализ

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [ ]:
df0 = pd.read_csv("/dataset.csv")
df0.info()
df0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790 entries, 0 to 14789
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        14790 non-null  int64  
 1   annual_income              14790 non-null  float64
 2   num_bank_acc               14790 non-null  int64  
 3   num_credit_card            14790 non-null  int64  
 4   interest_rate              14790 non-null  int64  
 5   delay_from_due_date        14790 non-null  int64  
 6   outstanding_debt           14790 non-null  float64
 7   credit_history_age         14790 non-null  float64
 8   installment_per_month      14790 non-null  float64
 9   monthly_balance            14790 non-null  float64
 10  payment_of_min_amount_Yes  14790 non-null  bool   
 11  Predicted Credit Score     14790 non-null  int64  
dtypes: bool(1), float64(5), int64(6)
memory usage: 1.3 MB


,age,annual_income,num_bank_acc,num_credit_card,interest_rate,delay_from_due_date,outstanding_debt,credit_history_age,installment_per_month,monthly_balance,payment_of_min_amount_Yes,Predicted Credit Score
0,23,19114.12,3,4,3,3,809.98,22.90,49.57,186.27,False,2
1,24,19114.12,3,4,3,3,809.98,22.10,49.57,361.44,False,2
2,28,34847.84,2,4,6,3,605.03,27.40,18.82,303.36,False,2
3,28,34847.84,2,4,6,3,605.03,27.50,18.82,452.30,False,2
4,55,30689.89,2,5,4,5,632.46,17.11,16.42,425.73,False,1


* **age**	Age of the individual.	int64
* **annual_income**	Annual income of the individual.	float64
* **num_bank_acc**	Number of bank accounts owned.	int64
* **num_credit_card**	Number of credit cards owned.	int64
* **interest_rate**	Interest rate of credit card.	float64
* **delay_from_due_date**	Delayed days from payment's due date.	int64
* **outstanding_debt**	Amount of outstanding debt.	float64
* **credit_history_age**	Credit history age.	float64
* **payment_of_min_amount**	Indicates if the minimum amount is paid.	bool
* **installment_per_month**	Monthly installment amount.	float64
* **monthly_balance**	Monthly balance	float64
* **credit_score**	Credit score.	int64

### Разделение на обучающую и тестовую выборки

In [ ]:
dfX = df0.drop(columns=["Predicted Credit Score"])
dfY = df0["Predicted Credit Score"]

In [ ]:
print("\n=======X=======\n")
dfX.info()
dfX.head()

print("\n=======Y=======\n")
dfY.info()
dfY.head()


=======X=======

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790 entries, 0 to 14789
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        14790 non-null  int64  
 1   annual_income              14790 non-null  float64
 2   num_bank_acc               14790 non-null  int64  
 3   num_credit_card            14790 non-null  int64  
 4   interest_rate              14790 non-null  int64  
 5   delay_from_due_date        14790 non-null  int64  
 6   outstanding_debt           14790 non-null  float64
 7   credit_history_age         14790 non-null  float64
 8   installment_per_month      14790 non-null  float64
 9   monthly_balance            14790 non-null  float64
 10  payment_of_min_amount_Yes  14790 non-null  bool   
dtypes: bool(1), float64(5), int64(5)
memory usage: 1.1 MB

=======Y=======

<class 'pandas.core.series.Series'>
RangeIndex: 14790 entries, 0 to 

,Predicted Credit Score
0,2
1,2
2,2
3,2
4,1


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(
    dfX, dfY, test_size=0.2, random_state=1)

In [ ]:
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)

(11832, 11)
(2958, 11)
(11832,)
(2958,)


### Бэггинг

In [ ]:
from sklearn.ensemble import BaggingClassifier


bagging_model = BaggingClassifier(random_state=1)
bagging_model.fit(xTrain, yTrain)
y_pred_bagging = bagging_model.predict(xTest)

### Cлучайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(xTrain, yTrain)
y_pred_rf = rf_model.predict(xTest)

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


ada_model = AdaBoostClassifier(random_state=1)
ada_model.fit(xTrain, yTrain)
y_pred_ada = ada_model.predict(xTest)

### Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


gb_model = GradientBoostingClassifier(random_state=1)
gb_model.fit(xTrain, yTrain)
y_pred_gb = gb_model.predict(xTest)

### Оценка качества

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate


# Метрики
bagging_accuracy = round(accuracy_score(yTest, y_pred_bagging), 3)
rf_accuracy = round(accuracy_score(yTest, y_pred_rf), 3)
ada_accuracy = round(accuracy_score(yTest, y_pred_ada), 3)
gb_accuracy = round(accuracy_score(yTest, y_pred_gb), 3)

bagging_precision = round(precision_score(yTest, y_pred_bagging, average='weighted'), 3)
rf_precision = round(precision_score(yTest, y_pred_rf, average='weighted'), 3)
ada_precision = round(precision_score(yTest, y_pred_ada, average='weighted'), 3)
gb_precision = round(precision_score(yTest, y_pred_gb, average='weighted'), 3)

bagging_recall = round(recall_score(yTest, y_pred_bagging, average='weighted'), 3)
rf_recall = round(recall_score(yTest, y_pred_rf, average='weighted'), 3)
ada_recall = round(recall_score(yTest, y_pred_ada, average='weighted'), 3)
gb_recall = round(recall_score(yTest, y_pred_gb, average='weighted'), 3)

bagging_f1 = round(f1_score(yTest, y_pred_bagging, average='weighted'), 3)
rf_f1 = round(f1_score(yTest, y_pred_rf, average='weighted'), 3)
ada_f1 = round(f1_score(yTest, y_pred_ada, average='weighted'), 3)
gb_f1 = round(f1_score(yTest, y_pred_gb, average='weighted'), 3)

# Табличка
data = [
    ['accuracy', bagging_accuracy, rf_accuracy, ada_accuracy, gb_accuracy],
    ['precision', bagging_precision, rf_precision, ada_precision, gb_precision],
    ['recall', bagging_recall, rf_recall, ada_recall, gb_recall],
    ['f1', bagging_f1, rf_f1, ada_f1, gb_f1]
]

headers = ['Метрика / Модель', 'Bagging', 'Random Forest', 'AdaBoost', 'Gradient Boosting']

print(tabulate(data, headers=headers, tablefmt='grid'))

+--------------------+-----------+-----------------+------------+---------------------+
| Метрика / Модель   |   Bagging |   Random Forest |   AdaBoost |   Gradient Boosting |
+====================+===========+=================+============+=====================+
| accuracy           |     0.907 |           0.929 |      0.764 |               0.842 |
+--------------------+-----------+-----------------+------------+---------------------+
| precision          |     0.906 |           0.929 |      0.769 |               0.842 |
+--------------------+-----------+-----------------+------------+---------------------+
| recall             |     0.907 |           0.929 |      0.764 |               0.842 |
+--------------------+-----------+-----------------+------------+---------------------+
| f1                 |     0.907 |           0.929 |      0.755 |               0.842 |
+--------------------+-----------+-----------------+------------+---------------------+
